# Checking accuracy of BNF code change maps

An issue was raised by one of the clinical informatician team members:

>_An interesting case from an ICB colleague who is trying to check whether there’s any prescribing of Reslizumab (in response to a Medicines Supply Notification)…
OpenPrescribing says it has been prescribed, ePACT says there is no prescribing.
When looking at the EPD and PCA data, I can’t see any entries for Reslizumab at all - even nationally. In both cases I’m comparing November data, although I’ve checked other time periods with the same result. So, how are we detecting prescribing of Reslizumab in OpenPrescribing? Worried we could be ‘hallucinating’ prescribing!
p.s. I have re-read the spot the difference blog, but I don’t think it can explain this particular case._

Reslizumab is a monoclonal antibody which is normally only prescribed by specialists, and therefore we wouldn't expect to see it in primary care prescribing.  However [OpenPrescribing shows over 15,000 items per year](https://openprescribing.net/analyse/#org=CCG&numIds=0304020Z0&denom=nothing&selectedTab=summary).  As the EPD data doesn't have any prescribing, the most likely candidate is the BNF maps supplied by the NHSBSA.  These maps describe changes to the BNF codes made in January every year, and the Bennett Institute uses these maps to normalise the BNF code to the most current version across all time periods.



It would be worth checking the maps supplied by the NHSBSA (which are stored in the OpenPrescribing [GitHub repo](https://github.com/ebmdatalab/openprescribing/tree/main/openprescribing/frontend/management/commands/presentation_replacements) against both the "normalised" and "raw" data, to see if there are any abnormalities.

In [2]:
#import required libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ebmdatalab import bq
import os
import requests
from io import StringIO

### Import map data from Github

In [7]:
res = requests.get('https://api.github.com/repos/ebmdatalab/openprescribing/contents/openprescribing/frontend/management/commands/presentation_replacements') #uses GitHub API to get list of all files listed in measure definitions - defaults to main branch
data = res.text #creates text from API result
github_df = pd.read_json(data) #turns JSON from API result into dataframe
display(github_df) # displays all available files

,name,path,sha,size,url,html_url,git_url,download_url,type,_links
0,2010.txt,openprescribing/frontend/management/commands/p...,41fea5049304d0606a27a426edc305738f84cdf1,2338,https://api.github.com/repos/ebmdatalab/openpr...,https://github.com/ebmdatalab/openprescribing/...,https://api.github.com/repos/ebmdatalab/openpr...,https://raw.githubusercontent.com/ebmdatalab/o...,file,{'self': 'https://api.github.com/repos/ebmdata...
1,2011.txt,openprescribing/frontend/management/commands/p...,413dcdd0391f9faa1c7564b303cea5adf7ad8c78,1234,https://api.github.com/repos/ebmdatalab/openpr...,https://github.com/ebmdatalab/openprescribing/...,https://api.github.com/repos/ebmdatalab/openpr...,https://raw.githubusercontent.com/ebmdatalab/o...,file,{'self': 'https://api.github.com/repos/ebmdata...
2,2012.txt,openprescribing/frontend/management/commands/p...,7534f93db2bcc6eaa2f32784f32fa87c33ad31aa,1976,https://api.github.com/repos/ebmdatalab/openpr...,https://github.com/ebmdatalab/openprescribing/...,https://api.github.com/repos/ebmdatalab/openpr...,https://raw.githubusercontent.com/ebmdatalab/o...,file,{'self': 'https://api.github.com/repos/ebmdata...
3,2013.txt,openprescribing/frontend/management/commands/p...,524fd0836a2c2f7e37285b48b51db24dfd193060,2299,https://api.github.com/repos/ebmdatalab/openpr...,https://github.com/ebmdatalab/openprescribing/...,https://api.github.com/repos/ebmdatalab/openpr...,https://raw.githubusercontent.com/ebmdatalab/o...,file,{'self': 'https://api.github.com/repos/ebmdata...
4,2014.txt,openprescribing/frontend/management/commands/p...,14349fc8e21ebd3808f73306d851acf2e7b88e52,1644,https://api.github.com/repos/ebmdatalab/openpr...,https://github.com/ebmdatalab/openprescribing/...,https://api.github.com/repos/ebmdatalab/openpr...,https://raw.githubusercontent.com/ebmdatalab/o...,file,{'self': 'https://api.github.com/repos/ebmdata...
5,2015.txt,openprescribing/frontend/management/commands/p...,be85a7dc63202c282939dfbb0d5180dcb0f270b9,1372,https://api.github.com/repos/ebmdatalab/openpr...,https://github.com/ebmdatalab/openprescribing/...,https://api.github.com/repos/ebmdatalab/openpr...,https://raw.githubusercontent.com/ebmdatalab/o...,file,{'self': 'https://api.github.com/repos/ebmdata...
6,2017.txt,openprescribing/frontend/management/commands/p...,f5479cdfedd9ef67ad219d9150ba742c37a1fc5c,3029,https://api.github.com/repos/ebmdatalab/openpr...,https://github.com/ebmdatalab/openprescribing/...,https://api.github.com/repos/ebmdatalab/openpr...,https://raw.githubusercontent.com/ebmdatalab/o...,file,{'self': 'https://api.github.com/repos/ebmdata...
7,2018.txt,openprescribing/frontend/management/commands/p...,da3cb110a9fa50f21fdf8ed475bba785016d40ec,6335,https://api.github.com/repos/ebmdatalab/openpr...,https://github.com/ebmdatalab/openprescribing/...,https://api.github.com/repos/ebmdatalab/openpr...,https://raw.githubusercontent.com/ebmdatalab/o...,file,{'self': 'https://api.github.com/repos/ebmdata...
8,2020.txt,openprescribing/frontend/management/commands/p...,d8ac4ef6e7bf8b1c0af63d0a9bc73d05bcd376f4,27424,https://api.github.com/repos/ebmdatalab/openpr...,https://github.com/ebmdatalab/openprescribing/...,https://api.github.com/repos/ebmdatalab/openpr...,https://raw.githubusercontent.com/ebmdatalab/o...,file,{'self': 'https://api.github.com/repos/ebmdata...
9,2021.txt,openprescribing/frontend/management/commands/p...,7d634ef27f284d666e84c35359915f6faa5aa346,736,https://api.github.com/repos/ebmdatalab/openpr...,https://github.com/ebmdatalab/openprescribing/...,https://api.github.com/repos/ebmdatalab/openpr...,https://raw.githubusercontent.com/ebmdatalab/o...,file,{'self': 'https://api.github.com/repos/ebmdata...


In [8]:
codes_df = pd.DataFrame() #creates blank dataframe
for row in github_df[github_df['name'].str.contains('.txt')].itertuples(index=True): #iterates through rows, and continues if file is .txt 
        url = (getattr(row, "download_url")) #gets URL from API request  
        year_df=pd.read_csv(url,sep='\t',header=None, names=['old_bnf_code', 'new_bnf_code']) # creates 2 columns from tab-separated txt file, and names the column header
        year_df['change_date'] = pd.to_datetime(row.name[:4], format='%Y').strftime('%Y-%m-%d') # adds the year of change from the file name in a yyyy-mm-dd format
        codes_df = pd.concat([codes_df,year_df], axis=0, ignore_index=True) # concatentates into single dataframe

In [10]:
display(codes_df)

,old_bnf_code,new_bnf_code,change_date
0,0905013A0BBAAA0,0905013A0BBAAAF,2010-01-01
1,140300000BKBHA0,140300000BKBHAE,2010-01-01
2,140300000BKBGA0,140300000BKBGAD,2010-01-01
3,0408010ACAAAAAA,40801020AAAHAH,2010-01-01
4,0408010ACAAABAB,40801020AAAIAI,2010-01-01
...,...,...,...
2894,190700000BBCZA0,0913161A0BFABAB,2023-01-01
2895,190700000BBDABP,0913161A0BDAAAF,2023-01-01
2896,191300000BMCHA0,0913171B0BBABAB,2023-01-01
2897,191300000BMCIA0,0913171B0BBAAAA,2023-01-01


### Create codelist to download data from BigQuery

In [15]:
bnf_code_list = codes_df['old_bnf_code'].astype(str).tolist() + codes_df['new_bnf_code'].astype(str).tolist() # create two lists (one for old BNF codes, one for new), and concatenate into one list
where_clause = "(BNF_CODE LIKE '" +"%' OR BNF_CODE LIKE '".join(bnf_code_list) + "')" # create "WHERE bnf_code LIKE 'x%' or bnf_code LIKE 'y%'"" format for use with BigQuery

### Get data from BigQuery

In [19]:
#this query downloads all data from the raw data with either old or new BNF codes
sql = f"""
SELECT
  month,
  bnf_code,
  bnf_name,
  items
FROM
  richard.all_prescribing_items
WHERE {in_clause}
  """
exportfile = os.path.join("..","data","items_df.csv")
items_df = bq.cached_read(sql, csv_path=exportfile, use_cache=True)
display(items_df)

,month,bnf_code,bnf_name,items
0,2014-03-01,090401000BBQZA0,TYR express20 oral powder 34g sachets,1
1,2014-03-01,0904010U0BDARAA,Ener-G gluten free flax loaf sliced,8
2,2014-03-01,090401000BBUBA0,Vital 1.5kcal liquid (3 flavours),610
3,2014-03-01,090402000BBTJA0,Fresubin 5kcal shot drink neutral,836
4,2014-03-01,091200000BEWDDG,Valupak Glucosamine sulfate 500mg tablets,367
...,...,...,...,...
257673,2021-03-01,090402000BBZVA0,Ensure Plus Commence liquid assorted,568
257674,2021-03-01,091000000BBWVBU,MacuLEH Light tablets,1
257675,2021-03-01,090402000BBWPA0,Meritene Energis Soup oral powder 50g sachets ...,213
257676,2021-03-01,140400030BDAAAD,VAQTA Paediatric vacc inj 0.5ml pre-filled syr...,226


In [131]:
#pd.options.mode.chained_assignment = None # suppress warning
#filtered_df['code_length'] = filtered_df['old_bnf_code'].astype(str).apply(len)

### Merge with BNF code change maps

In [24]:
key = items_df['bnf_code'].str.extract('^(' + '|'.join(codes_df['old_bnf_code']) + ')') # create key to allow partial match of codes to join
code_check_df = codes_df.merge(items_df.assign(key=key), left_on='old_bnf_code', right_on='key').drop('key', 1) # merge two dfs using the key above
display(code_check_df)

,old_bnf_code,new_bnf_code,change_date,month,bnf_code,bnf_name,items
0,190205200BBBDA0,190400000BBBRUI,2011-01-01,2010-09-01,190205200BBBDA0,After Bite_Insect Bite TT Pen,50
1,190205200BBBDA0,190400000BBBRUI,2011-01-01,2010-08-01,190205200BBBDA0,After Bite_Insect Bite TT Pen,41
2,190205200BBBDA0,190400000BBBRUI,2011-01-01,2010-10-01,190205200BBBDA0,After Bite_Insect Bite TT Pen,53
3,0106040M0BBAEAE,0106040M0BBAEAA,2011-01-01,2010-10-01,0106040M0BBAEAE,Movicol_Pdr Sach 13.9g (Choc),2615
4,0106040M0BBAEAE,0106040M0BBAEAA,2011-01-01,2010-08-01,0106040M0BBAEAE,Movicol_Pdr Sach 13.9g (Choc),2739
...,...,...,...,...,...,...,...
209850,191300000BMCUA0,0913421A0BBAAAA,2023-01-01,2018-08-01,191300000BMCUA0,Hydrafast pre-thickened fruit water 90ml pouches,2
209851,191300000BMCUA0,0913421A0BBAAAA,2023-01-01,2017-12-01,191300000BMCUA0,Hydrafast pre-thickened fruit water 90ml pouches,9
209852,191300000BMCUA0,0913421A0BBAAAA,2023-01-01,2018-02-01,191300000BMCUA0,Hydrafast pre-thickened fruit water 90ml pouches,2
209853,191300000BMCUA0,0913421A0BBAAAA,2023-01-01,2019-08-01,191300000BMCUA0,Hydrafast pre-thickened fruit water 90ml pouches,2


In [196]:
filtered_df = df3[df3['old_bnf_code'].apply(lambda x: len(str(x)) != 15)]

In [197]:
filtered_df.head()

,old_bnf_code,new_bnf_code,change_date,month,bnf_code,bnf_name,items
11,1306020K0,0501030Z0,2011-01-01,2010-09-01,1306020K0BBAAAA,Efracea_Cap 40mg M/R,80
12,1306020K0,0501030Z0,2011-01-01,2010-08-01,1306020K0AAAAAA,Doxycycline_Cap 40mg M/R,228
13,1306020K0,0501030Z0,2011-01-01,2010-10-01,1306020K0BBAAAA,Efracea_Cap 40mg M/R,58
14,1306020K0,0501030Z0,2011-01-01,2010-09-01,1306020K0AAAAAA,Doxycycline_Cap 40mg M/R,236
15,1306020K0,0501030Z0,2011-01-01,2010-10-01,1306020K0AAAAAA,Doxycycline_Cap 40mg M/R,252


In [198]:
df3['month'] = pd.to_datetime(df3['month'])

In [199]:
df3['change_date'] = pd.to_datetime(df3['change_date'])

In [216]:
filtered_df = df3[df3['month'] > df3['change_date']]

In [217]:
filtered_df.head(500)

,old_bnf_code,new_bnf_code,change_date,month,bnf_code,bnf_name,items
53312,0408010AHBBAGAG,Withdrawn (no new code),2013-01-01,2013-04-01,0408010AHBBAGAG,Vimpat_Syr 10mg/ml,2
53314,0408010AHBBAGAG,Withdrawn (no new code),2013-01-01,2013-09-01,0408010AHBBAGAG,Vimpat_Syr 10mg/ml,1
53318,0408010AHBBAGAG,Withdrawn (no new code),2013-01-01,2013-05-01,0408010AHBBAGAG,Vimpat_Syr 10mg/ml,2
53319,0408010AHBBAGAG,Withdrawn (no new code),2013-01-01,2013-08-01,0408010AHBBAGAG,Vimpat_Syr 10mg/ml,2
53321,0408010AHBBAGAG,Withdrawn (no new code),2013-01-01,2013-06-01,0408010AHBBAGAG,Vimpat_Syr 10mg/ml,6
...,...,...,...,...,...,...,...
61232,1306010ACBBAAAA,1306030B0BBAAAA,2018-01-01,2023-05-01,1306010ACBBAAAA,Aklief 50micrograms/g cream,31
61233,1306010ACBBAAAA,1306030B0BBAAAA,2018-01-01,2022-11-01,1306010ACBBAAAA,Aklief 50micrograms/g cream,4
61234,1306010ACBBAAAA,1306030B0BBAAAA,2018-01-01,2023-09-01,1306010ACBBAAAA,Aklief 50micrograms/g cream,76
61235,1306010ACBBAAAA,1306030B0BBAAAA,2018-01-01,2023-03-01,1306010ACBBAAAA,Aklief 50micrograms/g cream,17


In [226]:
result_df = filtered_df.groupby(['old_bnf_code', 'new_bnf_code', 'bnf_name', 'change_date']).agg({'items': 'sum', 'month': 'max'}).reset_index().sort_values(by='items', ascending=False).rename(columns={'month': 'latest_month'})

In [227]:
result_df.head(200)

,old_bnf_code,new_bnf_code,bnf_name,change_date,items,latest_month
1,0301040V0BBAAAA,0301020U0BBAAAA,Duaklir 340micrograms/dose / 12micrograms/dose...,2017-01-01,1086096,2023-11-01
0,0301040V0AAAAAA,0301020U0AAAAAA,Aclidinium brom 396mcg/dose / Formoterol 11.8m...,2017-01-01,94468,2023-11-01
3,0302000W0BBAAAA,0304020Z0BBAAAA,Enerzair Breezhaler 114microg / 46microg / 136...,2018-01-01,20518,2023-11-01
7,0604011L0CCAABM,0703010S0BBAAAA,Bijuve 1mg/100mg capsules,2014-01-01,8420,2023-11-01
6,0604011L0AABMBM,0703010S0AAAAAA,Estradiol 1mg / Progesterone 100mg capsules,2014-01-01,3037,2023-11-01
8,1306010ACAAAAAA,1306030B0AAAAAA,Trifarotene 50micrograms/g cream,2018-01-01,632,2023-11-01
9,1306010ACBBAAAA,1306030B0BBAAAA,Aklief 50micrograms/g cream,2018-01-01,573,2023-11-01
10,1404000X0AAAHAH,1404000AQAAAAAA,"Meningococcal polysacch A, C, W135 & Y conj va...",2017-01-01,105,2023-10-01
11,1404000X0BKAAAH,1404000AQBBAAAA,MenQuadfi vaccine solution for injection 0.5ml...,2017-01-01,91,2023-11-01
2,0302000W0AAAAAA,0304020Z0AAAAAA,Generic Enerzair Breezhaler 114/46/136microg i...,2018-01-01,82,2023-11-01


In [228]:
merged_df = pd.merge(result_df, items_df, left_on='new_bnf_code', right_on='bnf_code', how='inner')

In [229]:
merged_df.head()

,old_bnf_code,new_bnf_code,bnf_name_x,change_date,items_x,latest_month,month,bnf_code,bnf_name_y,items_y
0,0604011L0CCAABM,0703010S0BBAAAA,Bijuve 1mg/100mg capsules,2014-01-01,8420,2023-11-01,2019-02-01,0703010S0BBAAAA,Zoely 2.5mg/1.5mg tablets,190
1,0604011L0CCAABM,0703010S0BBAAAA,Bijuve 1mg/100mg capsules,2014-01-01,8420,2023-11-01,2021-12-01,0703010S0BBAAAA,Zoely 2.5mg/1.5mg tablets,293
2,0604011L0CCAABM,0703010S0BBAAAA,Bijuve 1mg/100mg capsules,2014-01-01,8420,2023-11-01,2023-05-01,0703010S0BBAAAA,Zoely 2.5mg/1.5mg tablets,326
3,0604011L0CCAABM,0703010S0BBAAAA,Bijuve 1mg/100mg capsules,2014-01-01,8420,2023-11-01,2017-10-01,0703010S0BBAAAA,Zoely 2.5mg/1.5mg tablets,222
4,0604011L0CCAABM,0703010S0BBAAAA,Bijuve 1mg/100mg capsules,2014-01-01,8420,2023-11-01,2014-05-01,0703010S0BBAAAA,Zoely 2.5mg/1.5mg tablets,153


In [235]:
wrong_code_df =  merged_df.groupby(['old_bnf_code', 'new_bnf_code', 'bnf_name_x', 'bnf_name_y','change_date']).agg({'items_x':'sum', 'items_y': 'sum', 'month': 'max'})

In [236]:
wrong_code_df.head(200)

items_x  \
old_bnf_code    new_bnf_code    bnf_name_x                                         bnf_name_y                                         change_date            
0604011L0AABMBM 0703010S0AAAAAA Estradiol 1mg / Progesterone 100mg capsules        Estradiol 1.5mg / Nomegestrol 2.5mg tablets        2014-01-01    361403   
                                                                                   Estradiol/Nomegestrol_Tab 1.5mg/2.5mg              2014-01-01      6074   
0604011L0CCAABM 0703010S0BBAAAA Bijuve 1mg/100mg capsules                          Zoely 2.5mg/1.5mg tablets                          2014-01-01   1001980   
                                                                                   Zoely_Tab 2.5mg/1.5mg                              2014-01-01     16840   
1306010ACAAAAAA 1306030B0AAAAAA Trifarotene 50micrograms/g cream                   Ivermectin 10mg/g cream                            2018-01-01     64464   
1306010ACBBAAAA 1306030B0BBAAAA Aklief 50micrograms/g cream                        Soolantra 10mg/g cream                             2018-01-01     58446   
1404000X0AAAHAH 1404000AQAAAAAA Meningococcal polysacch A, C, W135 & Y conj vac... Generic Bexsero vaccine inj 0.5ml pre-filled sy... 2017-01-01      5565   
1404000X0BKAAAH 1404000AQBBAAAA MenQuadfi vaccine solution for injection 0.5ml ... Bexsero vacc inj 0.5ml pre-filled syringes         2017-01-01     10829   

                                                                                                                                                   items_y  \
old_bnf_code    new_bnf_code    bnf_name_x                                         bnf_name_y                                         change_date            
0604011L0AABMBM 0703010S0AAAAAA Estradiol 1mg / Progesterone 100mg capsules        Estradiol 1.5mg / Nomegestrol 2.5mg tablets        2014-01-01      4666   
                                                                                   Estradiol/Nomegestrol_Tab 1.5mg/2.5mg              2014-01-01        49   
0604011L0CCAABM 0703010S0BBAAAA Bijuve 1mg/100mg capsules                          Zoely 2.5mg/1.5mg tablets                          2014-01-01     23342   
                                                                                   Zoely_Tab 2.5mg/1.5mg                              2014-01-01       234   
1306010ACAAAAAA 1306030B0AAAAAA Trifarotene 50micrograms/g cream                   Ivermectin 10mg/g cream                            2018-01-01    261887   
1306010ACBBAAAA 1306030B0BBAAAA Aklief 50micrograms/g cream                        Soolantra 10mg/g cream                             2018-01-01    202454   
1404000X0AAAHAH 1404000AQAAAAAA Meningococcal polysacch A, C, W135 & Y conj vac... Generic Bexsero vaccine inj 0.5ml pre-filled sy... 2017-01-01       286   
1404000X0BKAAAH 1404000AQBBAAAA MenQuadfi vaccine solution for injection 0.5ml ... Bexsero vacc inj 0.5ml pre-filled syringes         2017-01-01     38819   

                                                                                                                                                       month  
old_bnf_code    new_bnf_code    bnf_name_x                                         bnf_name_y                                         change_date             
0604011L0AABMBM 0703010S0AAAAAA Estradiol 1mg / Progesterone 100mg capsules        Estradiol 1.5mg / Nomegestrol 2.5mg tablets        2014-01-01  2023-11-01  
                                                                                   Estradiol/Nomegestrol_Tab 1.5mg/2.5mg              2014-01-01  2013-12-01  
0604011L0CCAABM 0703010S0BBAAAA Bijuve 1mg/100mg capsules                          Zoely 2.5mg/1.5mg tablets                          2014-01-01  2023-11-01  
                                                                                   Zoely_Tab 2.5mg/1.5mg                              2014-01-01  2013-12-01  
1306010ACAAAAAA 1306030B0AAAAAA